# Hartree Fock Approximation. 

### Structure of notebook

1. Parameters of the cattering region are defined.
2. Parameters of the fermi dirac function & Integration interval are defined.
3. Electron Density is self-consistently calculated
4. The magnetocurrent $\Delta I(m,V) = I(m) - I(-m)$ is calculated.
5. P value is calculated

## Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Path to modules

In [ ]:
import sys
sys.path.insert(0, '/Users/khhuisman/Documents/Jupyter_notebooks/Github_Coulomb_Paper_Collinear/Modules_Coulomb_Github')

# sys.path.insert(0, '<path_to_modules>')

## Scattering Region

In [ ]:
import ChiralChainModel_git
import Geometry_Git
Geometry_Git.plot3d_chain(a=1,c=1,M=1,N=8)

## Functions for electron density

In [ ]:
import handy_functions_coulomb as hfc
import Integration_HFA as Integration_method
import Currents_HF_git as Current_method

In [ ]:
import negf_git

# Energies to integrate over, Fermi Energy

In [ ]:
def plot_densities(nP_list_conv,nM_list_conv,V_list_convg,Hamiltonian0):
    dimlist = hfc.func_list_i(Hamiltonian0)
    plt.title('$n_{is}(m,V) - n_{i\overline{s}}(-m,V)$')
    n_list_total_convgM_swap = [hfc.pairwise_swap([ nM_list_conv[i][k] for k in dimlist]) for i in range(len(V_list_convg))
                              ]
    nP_list_plot =[ [nP_list_conv[i][k] for k in dimlist ] for i in range(len(V_list_convg)) ]


    plt.plot(V_list_convg,np.subtract(nP_list_plot,n_list_total_convgM_swap))
    plt.xlabel('Bias Voltage [eV] ')
    plt.ylabel('Electron Density')
    plt.ticklabel_format(style="sci", scilimits=(0,0))

    plt.show()

In [ ]:
def func_energies(Hamiltonian0,U,npoints,npoints_tail):
    '''
    Input:
    Hamiltonian0 = molecule without interaction (U=0)
    U = interaction strength
    npoints = number of energy points in window [emin,emax]
    Output:
    emax = upper bound of intergral
    emin = lowest eigenvalue of Hamiltonian0
    energiesreal = list of energies between [emin-1000,emax] (make sure the emax < ef +Vmax/2)
    eflist = list of fermi energies for which to calculate electron densities.
    '''
    evlist = np.linalg.eigh(Hamiltonian0)[0]
    e_lumo = evlist[int(Hamiltonian0.shape[0]/2)-1]
    e_homo = evlist[int(Hamiltonian0.shape[0]/2)]
    #Fermi Energy
    hl_gap = e_lumo - e_homo
    
    
    
    #lower,upper bound for Glesser function
    emin = np.round(int(10*min(evlist))/10 - 10,2) #lower bound for integrals
    emax = np.round(int(10*max(evlist))/10 + 10,2)   #lower bound for integrals
    
    
    energies_zero4000 = np.linspace(emin-5000,emin,npoints_tail) #In wide band limit the integrand has a long "tail"
   
    energiesreal_prime = np.linspace(emin,emax,npoints)
    
    energiesreal = np.unique(hfc.jointwolist(energies_zero4000,energiesreal_prime))
    
    
    ### Note to user: The symmetric point mentioned in paper is not EF = U/2 but EF = 0 in this case,because
    ### the energies of the Green's function are shifted as: G+(E) -> G+(E + U/2).
    eflist = [ abs(np.round( 0  + kappa*hl_gap/2,2)) for kappa in [0.0] ]
    

    
    return emin,emax,npoints,energiesreal,eflist


## Define Scattering Region

In [ ]:
Lm = 8             #number of sites 
chirality = True   #handedness of molecule 
N = Lm             # Number of sites within one winding
M = 1              # Number of windings
a = 1              # radius of helix
c = 1              # pitch of helix


epsilon = 0 # onsite energy
t = 2.4     # hopping paramter

# List of Spin-Orbit Coupling parameter
lambdalist = [
            (1*(10**-1))*t
#             ,(1*(10**-3))*t
            ]
# List of interaction strength
Ulist = [ 0.2*t
#          , 0.1*t
        ] 


#Gamma Matrix
kmax  = 4      # Number spin up + down sites coupled to each lead
gamma = 0.5    # coupling strength
pz    = 0.5    # magnetic polarization

pz_P = abs(pz)
pz_M = -abs(pz)


T = 300 # Kelvin Temperature of leads
betaL,betaR = negf_git.func_beta(T), negf_git.func_beta(T) # 1/ElectronVolt




# Bias window

In [ ]:
Vmax = 0.1 # Maximum bias voltage [eV]
dV   = 0.1 # stepsize
V_list_pos_bias,V_list_total = hfc.func_V_total(Vmax,dV)

# Self-Consistent criteria & Integration

Comment on the convergence: Hartree Fock converges well for small U i.e. U<t.

In [ ]:
tol                 = 1*10**-5 #tolerance on the electron densities
tol_nintegrand      = 10**-7   #cut-off for integral over Glesser function
max_iteration       = 900      #maximum number of iterations
npoints             = 10000    #npoints energy point for Glesser integral [emin,emax]
npoints_tail        = 12000    #npoints energy point for Glesser integral [emin2,emin]
alpha               = 0.0      #linear mixing paramters
npoints_current     = 2000     #number of points for Landauer-Büttiker formula

In [ ]:
for U in Ulist:
    for lambda1 in lambdalist:
        GammaR,GammaLP,GammaLM,Hamiltonian0,hamiltonian_shape= ChiralChainModel_git.system_hamiltonian0(Lm,
                                                                                        epsilon,t, 
                                                                                  lambda1,chirality,
                                                                                  a,c,M,N,
                                                                                    kmax,gamma,abs(pz))
        
            
            
        emin,emax,npoints,energiesreal,eflist = func_energies(Hamiltonian0,U,npoints,npoints_tail)
        emin2 = min(energiesreal)
        
        for ef in eflist:
                   
            print('ef = {},lambda1/t = {} ,U/t = {}'.format(ef,lambda1/t,U/t) )

In [ ]:


for U in Ulist:
    for lambda1 in lambdalist:
        GammaR,GammaLP,GammaLM,Hamiltonian0,hamiltonian_shape= ChiralChainModel_git.system_hamiltonian0(Lm,
                                                                                        epsilon,t, 
                                                                                  lambda1,chirality,
                                                                                  a,c,M,N,
                                                                                    kmax,gamma,abs(pz))
        
            
            
        emin,emax,npoints,energiesreal,eflist = func_energies(Hamiltonian0,U,npoints,npoints_tail)
        emin2 = min(energiesreal)
        
        for ef in eflist:
            
                   
            print('ef = {},lambda1/t = {} ,U/t = {}'.format(ef,lambda1/t,U/t) )



            paramterslist = [npoints_current,Lm,N,M,a,c,chirality,kmax,pz,gamma,T,alpha,t,U,lambda1,ef,tol, Vmax,len(V_list_total),emin2,emin,emax,npoints,npoints_tail,tol_nintegrand]
            nP_list_total, convglistP = Integration_method.self_consistent_trapz_PN(V_list_pos_bias,Vmax,
                                  max_iteration,
                                ef,
                                U,
                                Hamiltonian0,
                                GammaLP,GammaR, 
                                betaL, betaR,tol,energiesreal,
                                tol_nintegrand,alpha,plot_bool=False,trackbool=True)


            nM_list_total, convglistM = Integration_method.self_consistent_trapz_PN(V_list_pos_bias,Vmax,
                                  max_iteration,
                                ef,
                                U,
                                Hamiltonian0,
                                GammaLM,GammaR, 
                                betaL, betaR,tol,energiesreal,
                                tol_nintegrand,alpha,plot_bool=False,trackbool=False)



            V_list_convg,nP_list_conv,nM_list_conv    = Integration_method.converged_lists(V_list_total,
                                                                                      nP_list_total , convglistP,
                                                                                          nM_list_total, convglistM)





In [ ]:
plot_densities(nP_list_conv,nM_list_conv,V_list_convg,Hamiltonian0)

# Currents $ I(\pm m,V)$ 

In [ ]:
#Calculate Currents
IP_list = Current_method.calc_I_trapz(npoints_current,
                V_list_convg,ef,
              Hamiltonian0 ,
              GammaLP,GammaR,
              U,nP_list_conv,
              betaL,betaR)

IM_list = Current_method.calc_I_trapz(npoints_current,
            V_list_convg,ef,
          Hamiltonian0 ,
          GammaLM,GammaR,
          U,nM_list_conv,
          betaL,betaR)


In [ ]:
## Currents
plt.title('$E_F = {}, U/t = {}$, $\lambda /t = {}$'.format(ef, U/t,lambda1/t))
plt.plot(V_list_convg,IP_list)
plt.plot(V_list_convg,IM_list)
plt.xlabel('Bias Voltage [eV]')
plt.ylabel('Current [eV]')
plt.ticklabel_format(style="sci", scilimits=(0,0))
plt.show()

# Magnetocurrent $\Delta I(m,V)$ 

In [ ]:


### Magnetocurrent
Vprime_list,PC_list = Current_method.func_MR_list(IP_list,IM_list,V_list_convg)
dIlist = np.subtract(IP_list,IM_list) 

plt.title('$\Delta I(m,V)$: $E_F = {}, U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,lambda1/t))
plt.plot(V_list_convg,dIlist)
plt.xlabel('Bias Voltage [eV]')
plt.ylabel('Current [eV]')
plt.ticklabel_format(style="sci", scilimits=(0,0))

plt.show()

plt.title('$E_F = {}$, $U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,lambda1/t))
plt.plot(Vprime_list , PC_list )
plt.xlabel('Bias Voltage')
plt.ylabel('$P_C$ [%]')
plt.ticklabel_format(style="sci", scilimits=(0,0))

plt.show()

# 5. P Value

In [ ]:
import Pvalue

In [ ]:
plt.title('$E_F = {} $, $U/t = {}$, $\lambda /t = {}$'.format(ef,U/t,lambda1/t))
Vlist_prime,PJ_list = Pvalue.function_PvaluedI(V_list_convg,dIlist,22)
plt.plot(Vlist_prime,PJ_list)
plt.xlim(0,Vmax)
plt.ylim(-1-0.1,1+0.1)
plt.show()